In [1]:
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

import numpy as np
print(np.__version__)

import os

1.21.6


In [2]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [3]:
ls

 2LS/                                  media.ipynb
 BlazarDNN.ipynb                       model.png
 BlazarDNN_vecchio.ipynb               model_v2/
 blazar_model_colab.ipynb              model_v3/
 blazar_model_colab_v2.ipynb           model_v4/
 blazar_model_colab_v3.ipynb           model_v5/
 blazar_model_colab_v5.ipynb           model_v6/
 blazar_model_colab_v6.ipynb           model_v6_nokfold/
 blazar_model_colab_v6_nokfold.ipynb   model_voub/
 blazar_model_colab_v7_plot.ipynb      newfile.txt
'Copia di KerasSimple.ipynb'           nn_data.npz
'Copy of CNN tutorial.ipynb'           nn_data_v2.npz
'Copy of Jet_exploration.ipynb'        nn_data_v3.npz
 example_txs0506+056.ipynb             nn_data_voub.npz
 k_model_pot.py                        __pycache__/
'Lezione GPU.ipynb'                    TXS0506056.txt
 Librerie.ipynb


In [4]:
# open file ASCII whit sed data of txs0506+056 and prepare for the net
ul_blazars = 0
bad_words = ['; UPPER LIMIT','NAN', 'nan','NaN']

filee = 'TXS0506056.txt'

# deleting rows whit upper limit ora NAN
with open(filee) as oldfile, open('newfile.txt', 'w') as newfile:
  for line in oldfile:
    if not any(bad_word in line for bad_word in bad_words):
      newfile.write(line)
    else:
      ul_blazars += 1
freq_opt = []                                                   # refined freq-flux array initialization
flux_opt = []
freq, flux = np.loadtxt('newfile.txt', usecols=(0, 2), unpack=True)
try:
  #deleting strange measures
  strange_flux_index = [i for i in range(len(freq)) if flux[i]>= 1e-5 or flux[i]<0]
  freq = np.delete(freq,strange_flux_index)
  flux = np.delete(flux,strange_flux_index)
  # if the frequencies are repeated, I keep the higher flux measurement
  for i in range(len(freq)):
    if np.size(np.where(freq_opt == freq[i]))>0:
      if flux[i] > flux_opt[np.where(freq_opt == freq[i])]:
        flux_opt[np.where(freq_opt == freq[i])]=flux[i]
    else:
      freq_opt= np.append(freq_opt,freq[i])
      flux_opt= np.append(flux_opt,flux[i])
except TypeError:
  freq_opt= np.append(freq_opt,freq[i])                       # In case there is only one freq-flux measurement
  flux_opt= np.append(flux_opt,flux[i])                       # does not give an error but simply appends value

In [5]:
len(freq_opt)

120

In [ ]:
print(ul_blazars)

4


In [6]:
nn_data = np.load('nn_data_v3.npz')

set_freq = nn_data['nn_freq_data']

In [7]:
input = np.zeros((set_freq.shape[0],2))
for i in range(len(freq_opt)):
  idx = np.where(set_freq == freq_opt[i])
  input[idx,0] = flux_opt[i]
  input[idx,1]= 1                        # flag at 1 for real data in sed file

In [8]:
input.shape

(529, 2)

In [10]:
ninput = np.expand_dims(input,axis=0)
ninput.shape

(1, 529, 2)

In [13]:
ninput = input[np.newaxis, :]
ninput.shape

(1, 529, 2)

In [18]:
from keras import models

model=models.load_model('model_v6/model_3.h5')
probs = model.predict(ninput)

print(f'TXS 0506+056 is a blazar whit a probability of {probs}')

TXS 0506+056 is a blazar whit a probability of [[0.74973524]]
